In [ ]:
import pickle
import matplotlib.pyplot as plt
import pandas as pd
from my_functions import z_NB
import numpy as np

In [ ]:
nbs_list = [[1, 4], [4, 8], [8, 12], [12, 16], [16, 20], [20, 24]]

df = None
for [nb_min, nb_max] in nbs_list:
    foldername = '/home/alberto/cosmos/LAEs/Luminosity_functions'
    selname = f'LF_r17-24_nb{nb_min}-{nb_max}_ew30_ewoth100_nb_0.5'
    filename = f'{foldername}/{selname}/selection.npy'
    with open(filename, 'rb') as f:
        selection = pickle.load(f)
    selection.keys()

    # Make DataFrame
    if df is None:
        df = pd.DataFrame(data=selection)
    else:
        df = pd.concat([df, pd.DataFrame(data=selection)])
# Drop duplicates
df = df.drop_duplicates(subset='RA')
df.dtypes

In [ ]:
# Save to npy/selection.npy
df.to_csv('csv/selection.csv')

In [ ]:
# Drop some columns
for col in ['src', 'x_im', 'y_im', 'other_lines']:
    df = df.drop(col, axis=1)

# Convert tile_id into tile names
tile_dict = {
    2241: 'AEGIS001',
    2243: 'AEGIS002',
    2406: 'AEGIS003',
    2470: 'AEGIS004',
    2520: 'J-NEP'
}

def tile_id_to_name(id):
    return tile_dict[int(id)]

def dec_to_sex(alpha):
    alpha = alpha * 60 * 60
    m, s = divmod(alpha, 60)
    d, m = divmod(m, 60)
    return f'{d:0.0f}º{m:0.0f}``{s:0.1f}\''

def redshift(nb):
    return f'{z_NB(nb)[0]:0.2f}'

def substitute_nans_zspec(what):
    if np.isnan(what):
        return '-'
    else:
        return f'{np.abs(what):0.2f}'

def substitute_nans_spcl(what):
    if what == 'nan':
        return '-'
    else:
        return what

def format_puri(p):
    return f'{p:0.2f}'

nb_Arr = np.array(df['nb_sel']).astype(int)

df['tile_id'] = df['tile_id'].astype(int).apply(tile_id_to_name)
df['RA'] = df['RA'].apply(dec_to_sex)
df['DEC'] = df['DEC'].apply(dec_to_sex)
df['nb_sel'] = df['nb_sel'].apply(redshift)
df['SDSS_spCl'] = df['SDSS_spCl'].astype(str).apply(substitute_nans_spcl)
df['SDSS_zspec'] = df['SDSS_zspec'].apply(substitute_nans_zspec)
df['puri'] = df['puri'].apply(format_puri)

ew_np = df['EW_lya'].to_numpy()
ew_err_np = df['EW_lya_err'].to_numpy()
ew_str = np.zeros(ew_np.shape).astype(str)
for i, (ew, err) in enumerate(zip(ew_np, ew_err_np)):
    ew_str[i] = f'{ew:0.0f}$\pm${err:0.0f}'
df['EW_lya'] = ew_str
df = df.drop('EW_lya_err', axis=1)

L_np = df['L_lya'].to_numpy()
L_err_np = df['L_lya_err'].to_numpy()
L_str = np.zeros(L_np.shape).astype(str)
for i, (L, err) in enumerate(zip(L_np, L_err_np)):
    L_str[i] = f'{L:0.2f}$\pm${err:0.2f}'
df['L_lya'] = L_str
df = df.drop('L_lya_err', axis=1)

mag = np.array(df['r'])
df = df.drop('r', axis=1)

# Sort by tile_id
df = df.sort_values(['tile_id', 'L_lya'])
df.tail()

In [ ]:
df.to_csv('csv/selection_table.csv')

In [ ]:
L_lya = np.array([L_text[:5] for L_text in df['L_lya']]).astype(float)
zspec = np.array([0 if item == '-' else float(item) for item in df['SDSS_zspec']])
z_Arr = df['nb_sel'].to_numpy().astype(float)
EW_Arr = ew_np
puri = df['puri'].to_numpy().astype(float)
df.keys()

In [ ]:
xmsdss = pd.read_csv('csv/xmatch_sdss_dr12.csv', sep=',', header=1)
zph = xmsdss['zph'].to_numpy()
e_zph = xmsdss['e_zph'].to_numpy()

In [ ]:
from my_functions import count_true
sdss_mask = (df['SDSS_spCl'] != '-') 
puri_mask = (puri > 0.)
L_mask = (L_lya > 40)
cont_mask = sdss_mask & L_mask & puri_mask & (np.abs(zspec - z_Arr) > 0.3)
conf_mask = sdss_mask & L_mask & puri_mask & ~(np.abs(zspec - z_Arr) > 0.3)
N_cont = count_true(cont_mask)
N_confirmed = count_true(conf_mask)

perc = N_confirmed / (N_confirmed + N_cont)
perc_e = ((N_confirmed + N_cont)**-4 * (N_cont**2 * N_confirmed + N_confirmed**2 * N_cont))**0.5
print(f'Confirmed sources: {N_confirmed} / {(N_confirmed + N_cont)} ; p = {perc:0.2f}({perc_e:0.2f})')

avgpuri_total = np.mean(puri[L_mask & puri_mask])
avgpuri_sdss = np.mean(puri[sdss_mask & L_mask & puri_mask])
print(f'Average Total purity = {avgpuri_total:0.3f}')
print(f'Average SDSS purity = {avgpuri_sdss:0.3f}')

In [ ]:
from my_functions import central_wavelength, nb_fwhm


fig, ax = plt.subplots(figsize=(5, 2.5))

ax.plot(zspec[conf_mask], z_Arr[conf_mask],
        marker='o', ls='', c='green', markerfacecolor='none')
ax.plot(zspec[cont_mask], z_Arr[cont_mask],
        marker='o', ls='', c='red', markerfacecolor='none')

# lines
zsp_xx = np.linspace(0, 5, 50)
w_lya = 1215.67
w_CIV = 1549.48
w_CIII = 1908.73
w_MgII = 2799.12
ax.plot(zsp_xx, zsp_xx, ls='--', c='orange', label='Ly$\alpha$', zorder=-99)
ax.plot((zsp_xx + 1) * w_lya / w_CIV - 1, zsp_xx, 
        ls='--', c='dimgray', label='CIV', zorder=-99)
ax.plot((zsp_xx + 1) * w_lya / w_CIII - 1, zsp_xx, 
        ls='--', c='dimgray', label='CIII', zorder=-99)
ax.plot((zsp_xx + 1) * w_lya / w_MgII - 1, zsp_xx, 
        ls='--', c='dimgray', label='MgII', zorder=-99)

text_plot = [[0.6, 3.5, 'MgII'],
             [1.6, 3.5, 'CIII'],
             [2.3, 3.5, 'CIV'],
             [3.2, 3.5, r'Ly$\alpha$']]
for text in text_plot:
        ax.text(text[0], text[1], text[2], fontsize=9)

# ax.set_yscale('log')
ax.set_ylim(2, 4.1)
ax.set_xlim(0, 4)
ax.set_xlabel('SDSS $z_\mathrm{spec}$', fontsize=15)
ax.set_ylabel('$z_\mathrm{NB}$', fontsize=15)
ax.tick_params(direction='in', which='both')
ax.yaxis.set_ticks_position('both')
ax.xaxis.set_ticks_position('both')

fig.savefig('figures/zspec_vs_z_Arr.pdf', bbox_inches='tight', facecolor='w', pad_inches=0)
plt.show()

fig, ax = plt.subplots(figsize=(5, 2.5))

w_central = central_wavelength()
nb_fwhm_Arr = np.array(nb_fwhm(np.arange(60)))
z_min = (w_central[nb_Arr] - nb_fwhm_Arr[nb_Arr] * 0.5) / w_lya - 1
z_max = (w_central[nb_Arr] + nb_fwhm_Arr[nb_Arr] * 0.5) / w_lya - 1
yerr = (z_max - z_min) * 0.5

ax.errorbar(zspec[conf_mask], (z_Arr - zspec)[conf_mask], yerr=yerr[conf_mask],
            fmt='o', markerfacecolor='w', markeredgecolor='green',
            capsize=4, ecolor='green')
ax.axhline(0, ls='--', c='k', zorder=-99)

ax.set_ylim(-0.25, 0.25)
ax.set_ylabel('$z_\mathrm{NB} - z_\mathrm{spec}$', fontsize=15)
ax.set_xlabel('SDSS $z_\mathrm{spec}$', fontsize=15)
ax.tick_params(direction='in', which='both')
ax.yaxis.set_ticks_position('both')
ax.xaxis.set_ticks_position('both')

fig.savefig('figures/delta_zspec.pdf', bbox_inches='tight', facecolor='w', pad_inches=0)
plt.show()

In [ ]:
def do_this():
        fig, ax = plt.subplots(figsize=(6, 3.5))

        bins = np.linspace(1, 4, 20 + 1)
        bins_c = [bins[i: i + 2].sum() * 0.5 for i in range(len(bins) - 1)]
        bins_sep = bins_c[1] - bins_c[0]
        EW_mock_good_h = np.load(f'npy/Expected_EW_hist.npy')
        EW_mock_cont_h = np.load(f'npy/Cont_EW_hist.npy')

        EW_this_h = np.histogram(np.log10(EW_Arr), bins=bins)[0] / len(EW_Arr)

        ax.plot(bins_c + bins_sep * 0.5, EW_this_h, drawstyle='steps', color='dimgray',
                zorder=99, label='miniJPAS + J-NEP')

        ax.plot(bins_c, EW_mock_good_h + EW_mock_cont_h, color='orange', ls='--',
                label='Mock (full selection)')
        ax.plot(bins_c, EW_mock_good_h, color='red', ls=':', label='Mock (only LAEs)')

        ax.set_ylabel('Fraction of sources', fontsize=12)
        ax.set_xlabel(r'log EW$_0^{\mathrm{Ly}\alpha}$ ($\AA$)', fontsize=12)

        ax.legend(fontsize=12)

        fig.savefig(f'figures/EW_hist.pdf', bbox_inches='tight',
                facecolor='w', edgecolor='w')
        plt.show()

do_this()

In [ ]:
from load_jpas_catalogs import load_minijpas_jnep
pm_flx, pm_err, tile_id, pmra_sn, pmdec_sn, parallax_sn, starprob, _,\
        spCl, zsp, photoz, photoz_chi_best, photoz_odds, N_minijpas, x_im, y_im,\
        ra, dec =\
        load_minijpas_jnep((['minijpas', 'jnep']))

In [ ]:
from astropy.io import fits
filename = '/home/alberto/almacen/SDSS_spectra_fits/DR16/DR16Q_Superset_v3.fits'
with fits.open(filename) as fread:
    z_visual_SDSS = fread[1].data['Z_VI']

fig, ax = plt.subplots(figsize=(6, 4))

bins = np.linspace(0, 5, 30)
ax.hist(zsp, bins, label='miniJPAS Xmatch')
ax.hist(z_visual_SDSS, bins, histtype='step',
        color='k', label='DR16Q')

ax.set_xlabel('z spec', fontsize=12)
ax.set_ylabel('Number of sources', fontsize=12)
ax.legend(fontsize=12)
ax.set_yscale('log')
ax.set_ylim(1, 1e6)

plt.show()
